In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:19pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:16pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))


<b><font size="7" color="red">ch14. 웹데이터 수집</font></b>
# 1절. BEAUTIFULSOUP과 parser
'pip install bs4' 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [5]:
# 내 Local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청 처리하는 Lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [6]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/ch14_sample.html')
response

<Response [404]>

In [7]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')
    

접근을 못했습니다


In [8]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406 : get, post 오류

404

In [9]:
response.content # 바이너리 형식의 html 

b"[Errno 2] No such file or directory: '\\\\ai\\\\ch14_sample.html'"

In [10]:
response.content.decode('utf-8')

"[Errno 2] No such file or directory: '\\\\ai\\\\ch14_sample.html'"

In [11]:
response.text # HTML파일의 텍스트 내용

"[Errno 2] No such file or directory: '\\\\ai\\\\ch14_sample.html'"

In [12]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, #response.text
                    "html.parser")
# soup

ModuleNotFoundError: No module named 'bs4'

In [13]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

NameError: name 'soup' is not defined

In [11]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1') # h1태그를 list
print('els :', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [12]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [13]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [14]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [15]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class','css'}))
print('find(None) :', soup.find('a', {'class':'css'}))


select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기 (네이버증권 -> 시장지표_)
-https://finance.naver.com/marketindex/

In [16]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)애ㅔ 대한 크롤링 허용

In [17]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
#response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")



In [18]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / responseread().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

In [19]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_ = 'value')
    #print(price.text.repalce(',', ''))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1464.0, 949.11, 1695.82, 205.57, 153.98, 1.16, 1.318, 99.31, 61.04, 1704.69, 4116.3, 194893.24]


In [20]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1464.0, 949.11, 1695.82, 205.57, 153.98, 1.16, 1.318, 99.31, 61.04, 1704.69, 4116.3, 194893.24]


In [21]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [22]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [23]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

상승	상승	상승	상승	하락	상승	상승	하락	상승	상승	하락	상승	

In [24]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [25]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,464.00원 - 상승
일본 JPY(100엔) : 949.11원 - 상승
유럽연합 EUR : 1,695.82원 - 상승
중국 CNY : 205.57원 - 상승
달러/일본 엔 : 153.9800엔 - 하락
유로/달러 : 1.1600달러 - 상승
영국 파운드/달러 : 1.3180달러 - 상승
달러인덱스 : 99.3100 - 하락
WTI : 61.04달러 - 상승
휘발유 : 1704.69원 - 상승
국제 금 : 4116.3달러 - 하락
국내 금 : 194893.24원 - 상승


In [26]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,464.00원 -상승
일본 JPY(100엔) : 949.11원 -상승
유럽연합 EUR : 1,695.82원 -상승
중국 CNY : 205.57원 -상승
달러/일본 엔 : 153.9800엔 -하락
유로/달러 : 1.1600달러 -상승
영국 파운드/달러 : 1.3180달러 -상승
달러인덱스 : 99.3100 -하락
WTI : 61.04달러 -상승
휘발유 : 1704.69원 -상승
국제 금 : 4116.3달러 -하락
국내 금 : 194893.24원 -상승


### 2) 이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
````
11997회(2025년 11월 08일 추첨)
당첨번호 [1,5,7,26,28,43]
보너스 30
````


In [27]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [28]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [29]:
times = soup.select_one('div.win_result strong').text
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [30]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title   href
0 "한풀 꺾인 비트코인 https://daum.net/v/202511110094711892
```

In [31]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

items_find_list = [] #검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    #print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd

pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [32]:
# 방법2 : 불가
from urllib.request import urlopen
from urllib.parse import quote
word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = urlopen(url)

import pandas as pd



https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


In [33]:
items_find_list = [] #검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호','제목','링크'])

,번호,제목,링크
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [34]:

items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [35]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    print(keyword)
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [36]:
collect_news_list('비트코인', 3)

비트코인


[[20,
  ' 파생상품 대거 청산, 암호화폐 일제 하락…리플 5%↓(상보) ',
  'http://v.daum.net/v/20251112073111389'],
 [21,
  ' 투자 사기 후 영국으로 도주한 중국인… 법원 "비트코인 9조원 압수" ',
  'http://v.daum.net/v/20251112083740757'],
 [22,
  ' 코스피 4097.44에 하락 출발…환율 1461원 ',
  'http://v.daum.net/v/20251112090609808'],
 [23,
  ' 비트코인, 美 정부 셧다운 해제 관망에 10만6000달러선 [크립토브리핑] ',
  'http://v.daum.net/v/20251111103648808'],
 [24,
  ' 개미 투자자 ‘데이터 무장’ 돕는다…업비트, 데이터랩 ‘인사이트’ 기능 강화 ',
  'http://v.daum.net/v/20251112090600801'],
 [25,
  " 9조 비트코인 들고 런던으로 튄 '中 폰지의 여왕', 징역 11년8개월 ",
  'http://v.daum.net/v/20251112062715613'],
 [26,
  ' 美 ‘셧다운 해제 기대감’에 가상자산 오름세… 비트코인 10만6200달러 ',
  'http://v.daum.net/v/20251111083345066'],
 [27,
  ' 12만명 속여 ‘비트코인 9조’ 40대 중국여성…도망간 英서 징역 11년8개월 ',
  'http://v.daum.net/v/20251112064227765'],
 [28,
  ' "내년에도 비트코인·K-방산"…\'지정학적 위기\' 뜨는 자산 ',
  'http://v.daum.net/v/20251110095715363'],
 [29,
  ' 서울대병원, 첫 비트코인 기부 받았다…1억5700만원 병원발전기금으로 ',
  'http://v.daum.net/v/20251111170538874']]

In [37]:
import time
keywords = ['청바지','동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결괄르 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번쨰 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword,page))
        elif i == 1:
            result1.extend(collect_news_list(keyword,page))
        time.sleep(0.5)
    
        

===1번쨰 검색어 청바지 검색 결과 수집 중입니다===
청바지
청바지
청바지
===2번쨰 검색어 동대문 검색 결과 수집 중입니다===
동대문
동대문
동대문


In [38]:
result0_df = pd.DataFrame(result0, columns=['no','title','link'])
result1_df = pd.DataFrame(result1, columns=['no','title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 ‘핫플’ 됐다",http://v.daum.net/v/20251111111251709
1,1,"동대문구, 사회복지시설 종사자 독감 예방접종 비용 지원",http://v.daum.net/v/20251112081345203
2,2,"동대문구, ‘2025 아이돌봄서비스 평가’ A등급(우수) 선정",http://v.daum.net/v/20251111172914828
3,3,"노보텔 앰배서더 동대문, 연말 맞아 '페스티브 파티’ 패키지 출시",http://v.daum.net/v/20251111155426418
4,4,"동대문구, 11월 학부모·학생 맞춤형 교육 특강",http://v.daum.net/v/20251111151146411


In [39]:
result0_df.to_csv('data/ch14_{keywords{0}}.csv',index=False, encoding='cp949')
result1_df.to_csv('data/ch14_{keywords{1}}.csv',index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라어저를 통해 요청하는 것 처럼 보이기

In [40]:
# 방법2 :
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
# headers = {'User-Agent':
#           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers = headers)
request = Request(url)
request.add_header('User-Agent',
                  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


###4)

In [41]:
# 방법 1
import requests
url = 'https://www.melon.com/chart/hot100/index.htm'
melonpage = requests.get(url)
melonpage
soup = BeautifulSoup(melonpage.text, 'html.parser')
soup




In [42]:
# 방법2
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/hot100/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
#soup

In [43]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/hot100/index.htm'

headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}

melonpage  = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melponpage.content
                    'html.parser')
ranks = soup.select('td div.wrap.t_center > span.rank')

titles = soup.select('div.ellipsis.rank01 > span')

singers = soup.select('div.ellipsis.rank02 > span')

#[singer.text.strip()[:20] for singer in singers]

for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXX
2위 | 타임캡슐 - 다비치
3위 | 달리 표현할 수 없어요 - 로이킴
4위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
5위 | Good Goodbye - 화사 (HWASA)
6위 | IRIS OUT - Kenshi Yonezu
7위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)
8위 | Hollywood Action - BOYNEXTDOOR
9위 | XOXZ - IVE (아이브)
10위 | 한번 더 이별 - 이창섭
11위 | Rich Man - aespa
12위 | body - 다영 (DAYOUNG)
13위 | FOCUS - Hearts2Hearts (하츠투하츠
14위 | BBUU! - PLAVE
15위 | 순간을 영원처럼 - 임영웅
16위 | 봉숭아 - PLAVE
17위 | OVERDRIVE - TWS (투어스)
18위 | 들꽃이 될게요 - 임영웅
19위 | JANE DOE - Kenshi Yonezu, Hikar
20위 | 그댈 위한 멜로디 - 임영웅
21위 | 숨바꼭질 (Hide and Seek) - PLAVE
22위 | 비가 와서 - 임영웅
23위 | 가만히 눈을 감고 - DK(디셈버)
24위 | 답장을 보낸지 - 임영웅
25위 | 나였으면 - 나윤권, 도경수(D.O.)
26위 | 알겠어요 미안해요 - 임영웅
27위 | ULSSIGU - 임영웅
28위 | 바이, 썸머 - 아이유
29위 | Wonderful Life - 임영웅
30위 | 꿈의 버스 - DAY6 (데이식스)
31위 | 숨바꼭질 (Hide and Seek) - PLAVE
32위 | 돌아보지 마세요 - 임영웅
33위 | 나는야 HERO - 임영웅
34위 | 우리에게 안녕 - 임영웅
35위 | 당신을 위하여 - 이보람 (씨야)
36위 | 있잖아 - BOYNEXTDOOR
37위 | Be My Light - 이세계아이돌
38위 | 0+0 - 한로로
39위 | Nameless - 이

### 5) 네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수만큼

In [44]:

# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
# url = 'https://kin.naver.com/search/list.naver'
# params = {'query' : keyword}
# response = get(url, params=params)
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
response = get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup


200


In [45]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [46]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link  = item.attrs.get('href')
        items_list.append({
            'title': title,
            'link' : link
        })
df = pd.DataFrame(items_list)
print(df.shape)
df.sample()

(30, 2)


,title,link
10,제가 직접 쓴 자소서 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색(open API 사용 O)